## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Current Description
0,0,Wuzhou,100,CN,2022-07-17 17:43:49,93,23.4833,111.3167,79.11,0.69,overcast clouds
1,1,Nome,0,US,2022-07-17 17:43:50,59,64.5011,-165.4064,52.16,5.75,clear sky
2,2,Los Llanos De Aridane,0,ES,2022-07-17 17:43:50,81,28.6585,-17.9182,73.13,16.11,clear sky
3,3,Marion,100,US,2022-07-17 17:43:51,90,39.7684,-86.1555,75.70,17.27,moderate rain
4,4,Kimbe,96,PG,2022-07-17 17:43:51,93,-5.5502,150.1429,74.84,3.96,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for you trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for you trip?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Current Description
0,0,Wuzhou,100,CN,2022-07-17 17:43:49,93,23.4833,111.3167,79.11,0.69,overcast clouds
3,3,Marion,100,US,2022-07-17 17:43:51,90,39.7684,-86.1555,75.70,17.27,moderate rain
16,16,Morondava,20,MG,2022-07-17 17:43:55,51,-20.2833,44.2833,75.16,12.59,few clouds
21,21,Eyl,51,SO,2022-07-17 17:43:56,73,7.9803,49.8164,78.17,32.53,broken clouds
22,22,Thompson,40,CA,2022-07-17 17:40:06,61,55.7435,-97.8558,77.16,4.61,scattered clouds
23,23,Jamestown,100,US,2022-07-17 17:43:57,70,42.0970,-79.2353,78.39,6.91,overcast clouds
27,27,Kapaa,100,US,2022-07-17 17:43:58,81,22.0752,-159.3190,79.12,13.80,overcast clouds
31,31,Butaritari,99,KI,2022-07-17 17:43:59,74,3.0707,172.7902,81.77,10.40,overcast clouds
37,37,Hambantota,100,LK,2022-07-17 17:44:03,78,6.1241,81.1185,80.65,16.49,overcast clouds
42,42,Hobyo,41,SO,2022-07-17 17:44:04,77,5.3505,48.5268,77.13,31.16,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                254
City                   254
Cloudiness             254
Country                254
Date                   254
Humidity               254
Lat                    254
Lng                    254
Max Temp               254
Wind Speed             254
Current Description    254
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Cloudiness             0
Country                0
Date                   0
Humidity               0
Lat                    0
Lng                    0
Max Temp               0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country","Max Temp", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Wuzhou,CN,79.11,23.4833,111.3167,
3,Marion,US,75.70,39.7684,-86.1555,
16,Morondava,MG,75.16,-20.2833,44.2833,
21,Eyl,SO,78.17,7.9803,49.8164,
22,Thompson,CA,77.16,55.7435,-97.8558,
23,Jamestown,US,78.39,42.0970,-79.2353,
27,Kapaa,US,79.12,22.0752,-159.3190,
31,Butaritari,KI,81.77,3.0707,172.7902,
37,Hambantota,LK,80.65,6.1241,81.1185,
42,Hobyo,SO,77.13,5.3505,48.5268,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
 lat = row["Lat"]
lng = row["Lng"]
params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found. Skipping...")
        

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))